In [4]:
import pandas as pd
import numpy as np
df = pd.read_csv('sample_data/DZ_2.csv')
df.head()

,Отметка времени,Итоговая оценка за экзамен по ТОЭ,"С какого раза вы сдали экзамен по ТОЭ (если улучшали оценку, так же указать с какого раза получили желаемую оценку)","Регулярность посещения занятий. Оцените процент занятий, которые посетили в последнем семетре по ТОЭ (лекции+практика)?",Оценка по физике в школе,Оценка по физике в последнем семестре?,Оценка за ИДЗ по ТОЭ за семестр,Оценка по лабораторным ТОЭ,Вели конспект по занятиям? (лекции и/или практики),Выходили к доске на практике?,...,В течении предыдущего семестра были трудоустроены и/или проходили стажировку?,"Если работали/проходили стажировку, то какая ставка (загрузка):",Где проживаете?,"Есть кот дома? (в месте, где сейчас живете)",Есть родной брат/сестра?,Гражданство,Сколько в среднем играли в прошлом семестре? Укажите число в часах,Была ли стипендия в 3 семестре?,Оцените свое ментальное здоровье в прошлом семестре,"Оцените, насколько были истощены (уставши) к началу сессии"
0,2024/09/30 11:51:10 PM GMT+3,5.0,1.0,75,5,5.0,5.0,5.0,да,да,...,нет,0.00,общежитие 8,нет,да,KAZ,10,да,9,1
1,2024/09/30 11:53:32 PM GMT+3,3.0,1.0,75,4,5.0,4.0,5.0,да,нет,...,нет,0.00,дома,нет,да,RUS,20,нет,7,6
2,2024/09/30 11:53:33 PM GMT+3,3.0,1.0,75,5,4.0,5.0,5.0,да,нет,...,да,0.25,дома,да,да,RUS,168,да,7,3
3,2024/09/30 11:57:50 PM GMT+3,3.0,1.0,75,5,4.0,4.0,4.0,нет,нет,...,да,0.50,общежитие 8,нет,да,KAZ,18,нет,7,2
4,2024/09/30 11:59:56 PM GMT+3,4.0,1.0,100,4,3.0,4.0,5.0,нет,нет,...,нет,NaN,дома,нет,да,RUS,20,нет,10,5


In [5]:
import matplotlib.pyplot as plt
import pickle
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Редактирование таблицы

Завмена значений Да/Нет на численные, упорядочивание типа данных (преводим все в float), чистим ненужные столбцы.

In [13]:
df = df.replace({'да': 1, 'нет': 0})
df = df.astype(float, errors='ignore')
df = df.drop('Отметка времени', axis=1)
df = df.drop('Гражданство', axis=1)
df = df.drop('Где проживаете?', axis=1)
df = df.drop('Сколько в среднем играли в прошлом семестре? Укажите число в часах', axis=1)
df = df.fillna(0)

Запускаем корреляционную таблицу:

In [14]:
cm = df.corr()
tc = cm['Итоговая оценка за экзамен по ТОЭ'].sort_values(ascending=False)
print(tc)

Итоговая оценка за экзамен по ТОЭ                                                                                          1.000000
Оценка за ИДЗ по ТОЭ за семестр                                                                                            0.721130
Оценка по лабораторным ТОЭ                                                                                                 0.660584
Регулярность посещения занятий. Оцените процент занятий, которые посетили в последнем семетре по ТОЭ (лекции+практика)?    0.598033
Была ли стипендия в 3 семестре?                                                                                            0.431341
Выходили к доске на практике?                                                                                              0.344403
Вели конспект по занятиям? (лекции и/или практики)                                                                         0.335572
Оцените, насколько были истощены (уставши) к началу сессии                  

Удаляем бесполезные столбцы:

In [18]:
df = df.drop(columns=['Если работали/проходили стажировку, то какая ставка (загрузка):', 'Есть родной брат/сестра?', 'В течении предыдущего семестра были трудоустроены и/или проходили стажировку?', 'Оцените свое ментальное здоровье в прошлом семестре', 'Оценка по физике в школе'])

KeyError: "['Если работали/проходили стажировку, то какая ставка (загрузка):', 'Есть родной брат/сестра?', 'В течении предыдущего семестра были трудоустроены и/или проходили стажировку?', 'Оцените свое ментальное здоровье в прошлом семестре', 'Оценка по физике в школе'] not found in axis"

In [21]:
c2m = df.corr()
t2c = c2m['Итоговая оценка за экзамен по ТОЭ'].sort_values(ascending=False)
print(t2c)

Итоговая оценка за экзамен по ТОЭ                                                                                          1.000000
Оценка за ИДЗ по ТОЭ за семестр                                                                                            0.721130
Оценка по лабораторным ТОЭ                                                                                                 0.660584
Регулярность посещения занятий. Оцените процент занятий, которые посетили в последнем семетре по ТОЭ (лекции+практика)?    0.598033
Была ли стипендия в 3 семестре?                                                                                            0.431341
Выходили к доске на практике?                                                                                              0.344403
Вели конспект по занятиям? (лекции и/или практики)                                                                         0.335572
Оцените, насколько были истощены (уставши) к началу сессии                  

Принято выбирать стобцы с корреляцией выше 0.7, но таких у нас всего два, поэтому возьмем еще и 3 для большей выборки:

In [42]:
X = df[['Оценка за ИДЗ по ТОЭ за семестр ', 'Оценка по лабораторным ТОЭ',  'Регулярность посещения занятий. Оцените процент занятий, которые посетили в последнем семетре по ТОЭ (лекции+практика)?']]
y = df['Итоговая оценка за экзамен по ТОЭ']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [43]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [44]:
y_pred = model.predict(X_test)

In [45]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse:.2f}')
print(f'R-squared: {r2:.2f}')

MSE: 0.50
R-squared: 0.05
